In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten


In [ ]:
# Load the dataset
import pandas as pd
data = pd.read_csv('Crop_recommendation.csv')

In [ ]:
data.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [ ]:
# Encode the categorical labels
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

In [ ]:
# Separate features and target
X = data.drop('label', axis=1)
y = data['label']

In [ ]:
 #Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Reshape data to fit LSTM requirements: (samples, timesteps, features)
X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(units=50, input_shape=(1, X_train.shape[1])))
lstm_model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# train lstm Model
history_lstm = lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
44/44 [==============================] - 3s 23ms/step - loss: 3.0202 - accuracy: 0.1818 - val_loss: 2.9414 - val_accuracy: 0.3523
Epoch 2/50
44/44 [==============================] - 0s 6ms/step - loss: 2.8420 - accuracy: 0.4077 - val_loss: 2.7467 - val_accuracy: 0.4489
Epoch 3/50
44/44 [==============================] - 0s 6ms/step - loss: 2.6094 - accuracy: 0.4673 - val_loss: 2.4913 - val_accuracy: 0.5114
Epoch 4/50
44/44 [==============================] - 0s 6ms/step - loss: 2.3250 - accuracy: 0.5469 - val_loss: 2.2046 - val_accuracy: 0.5767
Epoch 5/50
44/44 [==============================] - 0s 5ms/step - loss: 2.0306 - accuracy: 0.6065 - val_loss: 1.9261 - val_accuracy: 0.6506
Epoch 6/50
44/44 [==============================] - 0s 3ms/step - loss: 1.7548 - accuracy: 0.6747 - val_loss: 1.6716 - val_accuracy: 0.6847
Epoch 7/50
44/44 [==============================] - 0s 5ms/step - loss: 1.5097 - accuracy: 0.7280 - val_loss: 1.4482 - val_accuracy: 0.7358
Epoch 8/50
44/44 [=

In [ ]:
# Evaluate the LSTM model
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_lstm, y_test)

14/14 [==============================] - 1s 2ms/step - loss: 0.1375 - accuracy: 0.9568


In [ ]:
# Reshape data to fit CNN-LSTM requirements: (samples, timesteps, features)
X_train_cnn_lstm = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_cnn_lstm = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Build the CNN-LSTM model
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
cnn_lstm_model.add(MaxPooling1D(pool_size=2))
cnn_lstm_model.add(LSTM(units=50))
cnn_lstm_model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
cnn_lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the CNN-LSTM model
history_cnn_lstm = cnn_lstm_model.fit(X_train_cnn_lstm, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/50
44/44 [==============================] - 3s 17ms/step - loss: 3.0129 - accuracy: 0.1314 - val_loss: 2.8999 - val_accuracy: 0.1108
Epoch 2/50
44/44 [==============================] - 0s 5ms/step - loss: 2.6228 - accuracy: 0.1747 - val_loss: 2.3314 - val_accuracy: 0.2926
Epoch 3/50
44/44 [==============================] - 0s 4ms/step - loss: 1.8942 - accuracy: 0.4744 - val_loss: 1.5123 - val_accuracy: 0.6222
Epoch 4/50
44/44 [==============================] - 0s 5ms/step - loss: 1.2061 - accuracy: 0.7045 - val_loss: 1.0040 - val_accuracy: 0.7727
Epoch 5/50
44/44 [==============================] - 0s 5ms/step - loss: 0.8387 - accuracy: 0.8295 - val_loss: 0.7534 - val_accuracy: 0.8210
Epoch 6/50
44/44 [==============================] - 0s 6ms/step - loss: 0.6351 - accuracy: 0.8693 - val_loss: 0.5800 - val_accuracy: 0.8551
Epoch 7/50
44/44 [==============================] - 0s 5ms/step - loss: 0.5068 - accuracy: 0.9020 - val_loss: 0.4738 - val_accuracy: 0.9233
Epoch 8/50
44/44 [=

In [ ]:
# Evaluate the CNN-LSTM model
cnn_lstm_loss, cnn_lstm_accuracy = cnn_lstm_model.evaluate(X_test_cnn_lstm, y_test)

14/14 [==============================] - 1s 2ms/step - loss: 0.1019 - accuracy: 0.9636
